# Bloque 3: Feature Engineering + Clustering DTW en VM

Este notebook convierte tu script de Python en un cuaderno de Jupyter.

In [ ]:
import subprocess
import sys

# === VALIDACIÓN Y INSTALACIÓN DE DEPENDENCIAS ===
def instalar_paquete(paquete, version=None):
    try:
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{paquete}=={version}"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", paquete])
    except Exception as e:
        print(f"❌ Error instalando {paquete}: {e}")

In [ ]:
# Forzar versiones compatibles si da error de numpy
instalar_paquete("numpy", "1.24.4")
instalar_paquete("scipy", "1.10.1")
instalar_paquete("scikit-learn", "1.3.2")
instalar_paquete("tslearn")
instalar_paquete("gcsfs")

In [ ]:
# === IMPORTACIONES ===
import pandas as pd
import numpy as np
import gc
import gcsfs
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset

In [ ]:
# === CONFIGURACIÓN DE BUCKET Y PROYECTO ===
BUCKET = 'bukeli'
PROYECTO = 'carbide-crowbar-463114-d5'
INPUT_PATH = f'gs://{BUCKET}/panel/df_panel.parquet'
OUTPUT_PATH = f'gs://{BUCKET}/features/df_panel_features.parquet'

In [ ]:
# === INICIALIZAR SISTEMA DE ARCHIVOS ===
fs = gcsfs.GCSFileSystem(project=PROYECTO)

# === CARGAR DATASET DESDE BUCKET ===
print("📥 Cargando df_panel.parquet desde bucket...")
with fs.open(INPUT_PATH, 'rb') as f:
    df_pred = pd.read_parquet(f)

In [ ]:
# === OPTIMIZACIÓN DE TIPOS ===
def optimize_dtypes(df):
    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    return df

# === DROPEAR COLUMNA FECHA SI EXISTE ===
df_pred = df_pred.drop(columns=['fecha'], errors='ignore')

In [ ]:
# ===  Mapeo y One-Hot de cat1 ===
df_prod_cat1 = (
    df_pred[['product_id','cat1']]
    .dropna(subset=['cat1'])
    .drop_duplicates(subset=['product_id'])
)
mapeo_cat1 = dict(zip(df_prod_cat1['product_id'], df_prod_cat1['cat1']))

df_pred['cat1'] = df_pred['cat1'].fillna(df_pred['product_id'].map(mapeo_cat1))

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat1_ohe = ohe.fit_transform(df_pred[['cat1']])
ohe_cols = ohe.get_feature_names_out(['cat1'])
cat1_df = pd.DataFrame(cat1_ohe, columns=ohe_cols, index=df_pred.index)

df_pred = pd.concat([df_pred, cat1_df], axis=1)
df_pred.drop(columns=['cat1'], inplace=True)


In [ ]:
# ===  Variables de calendario y estacionales adicionales ===
df_pred['periodo_dt']       = pd.to_datetime(df_pred['periodo'], format='%Y%m')
df_pred['year']             = df_pred['periodo_dt'].dt.year
df_pred['month']            = df_pred['periodo_dt'].dt.month
df_pred['days_in_month']    = df_pred['periodo_dt'].dt.days_in_month
df_pred['semester']         = ((df_pred['month'] - 1) // 6) + 1
df_pred['quarter']          = df_pred['periodo_dt'].dt.quarter
df_pred['month_q']          = ((df_pred['month'] - 1) % 3) + 1

first_period = df_pred['periodo_dt'].min()
df_pred['period_ordinal']   = (
    (df_pred['periodo_dt'].dt.year  - first_period.year)  * 12 +
    (df_pred['periodo_dt'].dt.month - first_period.month + 1)
)
N = df_pred['period_ordinal'].max()
df_pred['period_ordinal_sin'] = np.sin(2 * np.pi * df_pred['period_ordinal'] / N)
df_pred['period_ordinal_cos'] = np.cos(2 * np.pi * df_pred['period_ordinal'] / N)

df_pred['is_summer']   = df_pred['month'].isin([12,1,2]).astype(int)
df_pred['is_winter']   = df_pred['month'].isin([6,7,8]).astype(int)
df_pred['pre_march']   = (df_pred['month']==2).astype(int)
df_pred['pre_october'] = (df_pred['month']==9).astype(int)

# === FEATURES TEMPORALES ORIGINALES ===
df_pred['mes']           = df_pred['periodo'] % 100
df_pred['año']           = df_pred['periodo'] // 100
df_pred['mes_sin']       = np.sin(2 * np.pi * df_pred['mes'] / 12)
df_pred['mes_cos']       = np.cos(2 * np.pi * df_pred['mes'] / 12)
df_pred['trimestre']     = ((df_pred['mes'] - 1) // 3) + 1
df_pred['trimestre_sin'] = np.sin(2 * np.pi * df_pred['trimestre'] / 4)
df_pred['trimestre_cos'] = np.cos(2 * np.pi * df_pred['trimestre'] / 4)
df_pred['fin_año']       = (df_pred['mes'] >= 11).astype(int)
df_pred['inicio_año']    = (df_pred['mes'] <= 2).astype(int)

# === FEATURES DE PRODUCTO ===
prod_group = df_pred.groupby('product_id')['tn']
df_pred['tn_media_prod']       = prod_group.transform('mean')
df_pred['tn_max_prod']         = prod_group.transform('max')
df_pred['tn_min_prod']         = prod_group.transform('min')
df_pred['tn_mediana_prod']     = prod_group.transform('median')
df_pred['tn_volatilidad_prod'] = prod_group.transform(lambda x: x.shift(1).rolling(12, min_periods=3).std().fillna(0))
for w in [3,6,12]:
    df_pred[f'tn_tendencia_{w}m'] = prod_group.transform(
        lambda x: x.shift(1).rolling(w, min_periods=int(w/2))
                      .apply(lambda y: np.polyfit(np.arange(len(y)), y,1)[0]
                             if len(y)>1 and y.std()>0 else 0, raw=False
                      ).fillna(0)
    )

# === FEATURES DE CLIENTE ===
cust_group = df_pred.groupby('customer_id')['tn']
df_pred['tn_cliente_media_6m']    = cust_group.transform(lambda x: x.shift(1).rolling(6, min_periods=1).mean())
df_pred['tn_media_cliente']       = cust_group.transform('mean')
df_pred['tn_volatilidad_cliente'] = cust_group.transform(lambda x: x.shift(1).rolling(12, min_periods=3).std().fillna(0))
df_pred['tn_crecimiento_cliente'] = cust_group.transform(lambda x: (x.shift(1) - x.shift(13)) / (x.shift(13) + 0.001))

# === INTERACCIÓN CLIENTE-PRODUCTO ===
combo_group = df_pred.groupby(['product_id','customer_id'])['tn']
df_pred['consistencia_prod_cliente'] = combo_group.transform(lambda x: (x.shift(1)>0).rolling(6, min_periods=1).mean())
df_pred['ratio_actual_historico']     = combo_group.transform(lambda x: x / (x.shift(1).rolling(12, min_periods=1).mean() + 0.001))


In [ ]:
# ===  Rolling windows completos e indicadores ===
combo = df_pred.groupby(['product_id','customer_id'])['tn']
for window in range(2,37):
    df_pred[f'tn_roll{window}']      = combo.transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    df_pred[f'tn_rollstd{window}']   = combo.transform(lambda x: x.shift(1).rolling(window, min_periods=1).std().fillna(0))
    df_pred[f'tn_rollmin{window}']   = combo.transform(lambda x: x.shift(1).rolling(window, min_periods=1).min().fillna(0))
    df_pred[f'tn_rollmax{window}']   = combo.transform(lambda x: x.shift(1).rolling(window, min_periods=1).max().fillna(0))
    df_pred[f'tn_is_new_rollmax{window}'] = (df_pred['tn']==df_pred[f'tn_rollmax{window}']).astype(int)
    df_pred[f'tn_is_new_rollmin{window}'] = (df_pred['tn']==df_pred[f'tn_rollmin{window}']).astype(int)
    df_pred[f'tn_div_roll{window}']  = df_pred['tn'] / (df_pred[f'tn_roll{window}'] + 1e-6)

for lag in [1,2,6,12]:
    df_pred[f'tn_lag_{lag}'] = combo.shift(lag)

In [ ]:
# ===  Deltas, aceleraciones y slopes de tendencia ===
for lag in range(1,13):
    lagged = combo.shift(lag)
    df_pred[f'tn_delta{lag}']     = df_pred['tn'] - lagged
    df_pred[f'tn_rel_delta{lag}'] = df_pred[f'tn_delta{lag}'] / (lagged + 1e-6)

for lag in range(1,12):
    df_pred[f'tn_accel{lag}'] = df_pred[f'tn_delta{lag}'] - df_pred[f'tn_delta{lag+1}']

def _rolling_slope(x, window):
    arr = np.array(x)
    if len(arr) < 2: return 0.0
    idx = np.arange(len(arr))
    return float(np.polyfit(idx, arr, 1)[0])

for window in [2,3,6,9,12,18,24,36]:
    df_pred[f'tn_trend_slope{window}'] = combo.apply(
        lambda x: pd.Series(x.values).rolling(window, min_periods=1)
                                      .apply(lambda arr: _rolling_slope(arr[-window:], window), raw=True)
    ).explode().values

df_pred['tn_vs_ma12'] = df_pred['tn'] - df_pred['tn_roll12']

In [ ]:
# === Edad de producto/cliente, racha sin ventas y limpieza ===
prod_start = df_pred.groupby('product_id')['periodo_dt'].min().rename('prod_start')
cust_start = df_pred.groupby('customer_id')['periodo_dt'].min().rename('cust_start')
df_pred = df_pred.join(prod_start, on='product_id').join(cust_start, on='customer_id')

df_pred['prod_age'] = (
    (df_pred['periodo_dt'].dt.year  - df_pred['prod_start'].dt.year)*12 +
    (df_pred['periodo_dt'].dt.month - df_pred['prod_start'].dt.month)
)
df_pred['cust_age'] = (
    (df_pred['periodo_dt'].dt.year  - df_pred['cust_start'].dt.year)*12 +
    (df_pred['periodo_dt'].dt.month - df_pred['cust_start'].dt.month)
)

def calcular_racha_no_ventas(serie):
    racha, count = [], 0
    for v in serie:
        count = count + 1 if v == 1 else 0
        racha.append(count)
    return pd.Series(racha, index=serie.index)

df_pred = df_pred.sort_values(['product_id','customer_id','periodo_dt'])
df_pred['no_sale']        = (df_pred['tn'] == 0).astype(int)
df_pred['no_sale_streak']= df_pred.groupby(['product_id','customer_id'])['no_sale']\
    .apply(calcular_racha_no_ventas).reset_index(level=[0,1],drop=True)

df_pred.replace([np.inf, -np.inf], np.nan, inplace=True)
df_pred.drop(columns=['plan_precios_cuidados'], inplace=True, errors='ignore')

# === EVENTOS EXTERNOS Y DEMÁS FEATURES ORIGINALES ===
df_pred['evento_agosto2019']         = (df_pred['periodo'] == 201906).astype(int)
df_pred['evento_crisis_post_paso']   = df_pred['periodo'].isin([201906,201907]).astype(int)
df_pred['evento_control_precios_2020']= (df_pred['periodo'] >= 201911).astype(int)
df_pred['es_precios_cuidados']       = df_pred['plan_precios_cuidados'].astype(int)

df_pred['demanda_latente']           = ((df_pred['cust_request_tn']>0)&(df_pred['tn']==0)).astype(int)
df_pred['demanda_latente_3m_prod']   = df_pred.groupby('product_id')['demanda_latente']\
    .transform(lambda x: x.shift(1).rolling(3, min_periods=1).sum())

df_pred['stock_final_lag1']          = df_pred.groupby(['product_id','customer_id'])['stock_final'].shift(1)
df_pred['cust_request_tn_lag1']      = df_pred.groupby(['product_id','customer_id'])['cust_request_tn'].shift(1)
df_pred['cust_request_qty_lag1']     = df_pred.groupby(['product_id','customer_id'])['cust_request_qty'].shift(1)

df_pred['tn_crecimiento_3m']         = df_pred.groupby(['product_id','customer_id'])['tn']\
    .transform(lambda x: (x.shift(1)-x.shift(4))/(x.shift(4)+0.001))
df_pred['tn_crecimiento_6m']         = df_pred.groupby(['product_id','customer_id'])['tn']\
    .transform(lambda x: (x.shift(1)-x.shift(7))/(x.shift(7)+0.001))
df_pred['tn_std_3m']                 = df_pred.groupby(['product_id','customer_id'])['tn']\
    .transform(lambda x: x.shift(1).rolling(3, min_periods=1).std())
df_pred['tn_std_6m']                 = df_pred.groupby(['product_id','customer_id'])['tn']\
    .transform(lambda x: x.shift(1).rolling(6, min_periods=1).std())

primera_fecha_prod = df_pred.groupby('product_id')['periodo'].transform('min')
df_pred['antiguedad_producto_meses']= df_pred['periodo'] - primera_fecha_prod
df_pred['producto_preexistente']     = (primera_fecha_prod == df_pred['periodo'].min()).astype(int)


In [ ]:
# === GUARDAR EN GCS ===OPTIMIZACIÓN FINAL Y EXPORTACIÓN ===

df_pred = optimize_dtypes(df_pred)
# (Opcional) eliminar columna temporal si existe
df_pred = df_pred.drop(columns=['fecha'], errors='ignore')

print("💾 Guardando df_panel_features.parquet en bucket...")
with fs.open(OUTPUT_PATH, 'wb') as f:
    df_pred.to_parquet(f, index=False)

print("✅ Feature engineering + clustering DTW terminado. Registros:", df_pred.shape[0])
print("Columnas:", list(df_pred.columns))
gc.collect()